Import libraries for the BERTopic topic modelling process

In [1]:
import pickle
import pandas as pd
import cufflinks as cf

import plotly.express as px
from plotly.offline import init_notebook_mode, iplot
from plotly.graph_objs import *

from ipywidgets import FloatProgress
from bertopic import BERTopic
from natsort import natsorted

cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

/Users/aneir1/.local/share/virtualenvs/lda_topic_models-22hpaXJ5/lib/python3.9/site-packages/umap/distances.py:1063: NumbaDeprecationWarning:

The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.

/Users/aneir1/.local/share/virtualenvs/lda_topic_models-22hpaXJ5/lib/python3.9/site-packages/umap/distances.py:1071: NumbaDeprecationWarning:

The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.

/Users/aneir1/.local/share/virtualenvs

Function to take dataframe and topic model. Takes date column and sentence text column and changes them into lists, applies topic model to the sentence list, then creates dictionary with topics as keys and a list of sentences for that topic as values. 

The sentence list, topics and date list are used to make a visualisation of topic frequency over time, with the output changed to scattered points rather than line.

Returns the dictionary of topics/sentences and the visualisation as tuple.

In [2]:
def bertopic_time(dataframe, topic_model):
    sent_list = dataframe["text"].to_list()
    date_list = dataframe["date"].to_list()
    topics, probs = topic_model.fit_transform(sent_list)
    
    topic_docs = {topic: [] for topic in set(topics)}
    for topic, doc in zip(topics, sent_list):
        topic_docs[topic].append(doc)
    
    topics_over_time = topic_model.topics_over_time(docs=sent_list, 
                                                timestamps=date_list, 
                                                global_tuning=True, 
                                                evolution_tuning=True, 
                                                nr_bins=20,
                                              )
    fig = topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=15, height=500, width=1000)
    fig.update_layout(yaxis_title = "Count")
    for trace in fig.data:
        trace.update(mode="markers")
    return (fig, topic_docs)

Function which takes dataframe and the tuple created by bertopic_time function and converts into a dataframe with the topics from the visualisation, which are limited to the top n topics in the dataset, in a new "topics" column. This new dataframe contains the topics from the visualisation, not all generated topics. Created for social network visualisations for each topic in notebook four.

First divides tuple into visualisation and topic/sentence list dictionary. For the latter, removes duplicates from sentence lists using set and reconverts to list.

Searches dataframe for sentence list for each topic. If the sentence is in a row, adds that row to a new dataframe with a new "topic" column containing the topic number. Append dataframe to a list of dataframes for all topics and then concatenates these into one dataframe.

Take the data from the visualisation and use this to get topic names and numbers for visual in ordered lists. Create final dataframe exclusively for the "top topics" contained in the visualisation with the topic names from the visualisation, replacing topic numbers created above.

Returns dataframe for the visualisation topics which includes topic names for each sentence.

In [3]:
def doc_topics_maker(dataframe, vis_obj):
    visual = vis_obj[0]
    doc_topics = vis_obj[1]
    doc_topics = dict(zip(doc_topics.keys(), map(set, doc_topics.values())))
    for key, value in doc_topics.items():
        doc_topics[key] = list(value)

    df_list = []
    for key, value in doc_topics.items():
        mask = dataframe["text"].isin(value)
        top_docs_df = dataframe.loc[mask]
        top_docs_df["topic"] = key
        df_list.append(top_docs_df)
    doc_topics_df = pd.concat(df_list)

    topic_names = []
    topic_nums = []
    topic_range = len(visual.data)
    for i in range(topic_range):
        topic_name = visual.data[i].name
        topic_names.append(topic_name)
        topic_nums.append(i)
    topic_names = natsorted(topic_names)

    doc_topics_df = doc_topics_df.loc[doc_topics_df["topic"].isin(topic_nums)]
    doc_topics_df = doc_topics_df.replace(topic_nums, topic_names)
    return doc_topics_df

Function which takes dataframe and topic model, extracts sentences from dataframe and runs through topic model, returns dendrogram of clusters for most prominent topics.

In [4]:
def bertopic_tree(dataframe, topic_model, topics_num=15):
    sent_list = dataframe["text"].to_list()
    topics, probs = topic_model.fit_transform(sent_list)
    fig = topic_model.visualize_hierarchy(top_n_topics=topics_num)
    return fig

Function which counts the number of letters in each year across the corpus, returns series for number of letters per year.

In [5]:
def year_counts(dataframe):
    dataframe["year"] = dataframe["date"].dt.year
    year_counts = dataframe.groupby(["year"]).size()
    return year_counts

Function which takes the year counts series for letters per year and converts into a bar plot.

In [6]:
def year_counts_plot(year_counts):
    year_counts_plot = px.bar(year_counts, 
                          title="Letter Count by Year",
                          labels= {
                              "value":"Count",
                              "year":"Year"
                          }
                         )
    year_counts_plot.update_layout(showlegend=False)
    return year_counts_plot

Function which creates a topic frequency over time graph, omitting outlying years with small numbers of letters and large numbers of letters. The function takes a dataframe, year counts series of letters per year and parameters with amendable default values for higher and lower limits of letters per year.

First the function creates the year window by removing high and low frequency year letters from the year counts series, then uses this year window to find items in the main dataframe which match it. Finally, it sends the resulting dataframe to bertopic_time for visualisation of the data over time and returns this visualisation.

In [7]:
def year_window_vis(dataframe, year_counts, topic_model, low_lim=20, high_lim=150):
    year_counts_window = year_counts[(year_counts > low_lim) & (year_counts < high_lim)]
    year_window = year_counts_window.index.tolist()
    year_win_df = dataframe[dataframe["date"].dt.year.isin(year_window)]
    year_win_vis_obj = bertopic_time(year_win_df, topic_model)
    year_win_vis = year_win_vis_obj[0]
    return year_win_vis

Import dataframes of clean data created in notebook zero, one with letters divided into sentences and the other with letter texts as one string.

In [8]:
sent_df = pd.read_pickle("pickle/henslow_sentences.pkl")
df = pd.read_pickle("pickle/henslow_texts.pkl")

Remove rows with empty text columns from the dataframe. This can happen because of the removal of parts of text during the data cleaning process.

In [9]:
sent_df = sent_df[sent_df["text"].str.len() != 0]
sent_df.reset_index(inplace=True, drop=True)
sent_df["text"] = sent_df["text"].str.join(" ")
sent_df

,letter,date,sender,recipient,text
0,letters_1.xml,1820-04-24,"Sowerby, James","Henslow, J. S.",mead lambeth april fossil
1,letters_1.xml,1820-04-24,"Sowerby, James","Henslow, J. S.",sedgwick favour amm
2,letters_1.xml,1820-04-24,"Sowerby, James","Henslow, J. S.",thank
3,letters_1.xml,1820-04-24,"Sowerby, James","Henslow, J. S.",clark lecture satr mot
4,letters_1.xml,1820-04-24,"Sowerby, James","Henslow, J. S.",iron pupil
...,...,...,...,...,...
9126,letters_1252.xml,1828-11-11,"Yarrell, William","Henslow, J. S.",london novr bennett act subcommittee managemen...
9127,letters_1252.xml,1828-11-11,"Yarrell, William","Henslow, J. S.",advantage catalogue collection bird
9128,letters_1252.xml,1828-11-11,"Yarrell, William","Henslow, J. S.",care leadbeater
9129,letters_1252.xml,1828-11-11,"Yarrell, William","Henslow, J. S.",refer case bird join jenyns promise


Use boolean maak to create dataframe restricted to 1920s for use in visualisation.

In [10]:
mask_20 = (sent_df["date"] > "1819/12/31") & (sent_df["date"] < "1829/12/31")
sent_df_year_restrict_20 = sent_df.loc[mask_20]
sent_df_year_restrict_20

,letter,date,sender,recipient,text
0,letters_1.xml,1820-04-24,"Sowerby, James","Henslow, J. S.",mead lambeth april fossil
1,letters_1.xml,1820-04-24,"Sowerby, James","Henslow, J. S.",sedgwick favour amm
2,letters_1.xml,1820-04-24,"Sowerby, James","Henslow, J. S.",thank
3,letters_1.xml,1820-04-24,"Sowerby, James","Henslow, J. S.",clark lecture satr mot
4,letters_1.xml,1820-04-24,"Sowerby, James","Henslow, J. S.",iron pupil
...,...,...,...,...,...
9126,letters_1252.xml,1828-11-11,"Yarrell, William","Henslow, J. S.",london novr bennett act subcommittee managemen...
9127,letters_1252.xml,1828-11-11,"Yarrell, William","Henslow, J. S.",advantage catalogue collection bird
9128,letters_1252.xml,1828-11-11,"Yarrell, William","Henslow, J. S.",care leadbeater
9129,letters_1252.xml,1828-11-11,"Yarrell, William","Henslow, J. S.",refer case bird join jenyns promise


Use boolean maak to create dataframe restricted to the period 1945 to 1957 for use in visualisation.

In [11]:
mask_late = (sent_df["date"] > "1844/12/31") & (sent_df["date"] < "1857/12/31")
sent_df_year_restrict_late = sent_df.loc[mask_late]
sent_df_year_restrict_late

,letter,date,sender,recipient,text
2605,letters_229.xml,1845-01-19,"Maund, Benjamin","Henslow, J. S.",bromsgrove january professor copy dict
2606,letters_229.xml,1845-01-19,"Maund, Benjamin","Henslow, J. S.",claim
2607,letters_229.xml,1845-01-19,"Maund, Benjamin","Henslow, J. S.",primula vulgaris elatior veris variety specie ...
2608,letters_229.xml,1845-01-19,"Maund, Benjamin","Henslow, J. S.",note evidence notice
2609,letters_229.xml,1845-01-19,"Maund, Benjamin","Henslow, J. S.",botanic_garden scientist variety williams pitm...
...,...,...,...,...,...
9109,letters_1250.xml,1848-08-21,"Henslow, J. S.","Ransome, George",glass tube
9110,letters_1250.xml,1848-08-21,"Henslow, J. S.","Ransome, George",sphaeria attack caterpillars fructifie neck
9111,letters_1250.xml,1848-08-21,"Henslow, J. S.","Ransome, George",hint sort book
9112,letters_1250.xml,1848-08-21,"Henslow, J. S.","Ransome, George",lindley vegetable kingdom


Create dataframes restricted to correspondence sent by Henslow and correspondence sent to him for visualisations.

In [12]:
henslow_sent_df = sent_df.loc[sent_df["sender"] == "Henslow, J. S."]
henslow_df = df.loc[df["sender"] == "Henslow, J. S."]
non_henslow_sent_df = sent_df.loc[sent_df["sender"] != "Henslow, J. S."]
non_henslow_df = df.loc[df["sender"] != "Henslow, J. S."]

Initialize BERTopic model and insert parameters, including input model and minimum topic number

In [13]:
topic_model = BERTopic(embedding_model="all-MiniLM-L6-v2", nr_topics="auto", min_topic_size=15)

Create visualisation of topics over time for all input data using bertopic_time function.

In [14]:
all_vis_obj = bertopic_time(sent_df, topic_model)
all_vis = all_vis_obj[0]
all_vis

Create visualisation of topics over time for 1920s letters using bertopic_time function.

In [15]:
sent_df_year_restrict_20_vis_obj = bertopic_time(sent_df_year_restrict_20, topic_model)
sent_df_year_restrict_20_vis = sent_df_year_restrict_20_vis_obj[0]
sent_df_year_restrict_20_vis

Create visualisation of topics over time for later-period letters using bertopic_time function.

In [16]:
sent_df_year_restrict_late_obj = bertopic_time(sent_df_year_restrict_late, topic_model)
sent_df_year_restrict_late_vis = sent_df_year_restrict_late_obj[0]
sent_df_year_restrict_late_vis

Use doc_topics_maker function to create dataframes for visualisations, limiting the dataframe to the topics contained within each visualisation and showing the topic names taken from the visualisation for each row. These visualisations are created for use with the social network visualisations in notebook four.

In [17]:
doc_topics_df = doc_topics_maker(sent_df, all_vis_obj)
doc_topics_df

,letter,date,sender,recipient,text,topic
8,letters_1.xml,1820-04-24,"Sowerby, James","Henslow, J. S.",martin,0_trin_henry_wilson_john_phillip
27,letters_2.xml,1821-11-15,"Clarke, E. D.","Henslow, J. S.",clarke,0_trin_henry_wilson_john_phillip
55,letters_6.xml,1822-07-02,"Curtis, John","Henslow, J. S.",bearer dr radius leipsic england man science s...,0_trin_henry_wilson_john_phillip
65,letters_8.xml,1823-04-02,"Henslow, J. S.","Jenyns, Leonard",suspect pera,0_trin_henry_wilson_john_phillip
108,letters_13.xml,1823-10-21,"Henslow, J. S.","Winch, N. J.",hodgson,0_trin_henry_wilson_john_phillip
...,...,...,...,...,...,...
6815,letters_947.xml,1826-07-02,"Palmerston, Lord","Henslow, J. S.",oclock voter,14_election_vote_voter_poll
6847,letters_956.xml,1826-06-19,"Loyd, S.","Henslow, J. S.",vote absence result opinion university,14_election_vote_voter_poll
8392,letters_1175.xml,1845-12-30,"Bright, John","Henslow, J. S.",west election probability walkover morpeth,14_election_vote_voter_poll
8663,letters_1205.xml,1826-02-14,"Henslow, J. S.","Dale, J. C.",cambridge feby dale election,14_election_vote_voter_poll


In [18]:
doc_topics_df_20s = doc_topics_maker(sent_df, sent_df_year_restrict_20_vis_obj)
doc_topics_df_20s

,letter,date,sender,recipient,text,topic
2,letters_1.xml,1820-04-24,"Sowerby, James","Henslow, J. S.",thank,0_palmerston_intention_morning_letter
7,letters_1.xml,1820-04-24,"Sowerby, James","Henslow, J. S.",punctatus,0_palmerston_intention_morning_letter
8,letters_1.xml,1820-04-24,"Sowerby, James","Henslow, J. S.",martin,0_palmerston_intention_morning_letter
11,letters_1.xml,1820-04-24,"Sowerby, James","Henslow, J. S.",scoticu,0_palmerston_intention_morning_letter
12,letters_1.xml,1820-04-24,"Sowerby, James","Henslow, J. S.",flattish,0_palmerston_intention_morning_letter
...,...,...,...,...,...,...
6902,letters_965.xml,1827-01-23,"Greville, R. K.","Henslow, J. S.",duplicate moss trading collection,14_collection_cryptogamia_thing_instance
7383,letters_1021.xml,1825-09-15,"Radius, Justus","Henslow, J. S.",cryptogamic,14_collection_cryptogamia_thing_instance
7418,letters_1022.xml,1828-12-24,"Radius, Justus","Henslow, J. S.",collection,14_collection_cryptogamia_thing_instance
7517,letters_1033.xml,1845-11-24,"Hooker, J. D.","Henslow, J. S.",collection,14_collection_cryptogamia_thing_instance


In [19]:
doc_topics_df_late = doc_topics_maker(sent_df, sent_df_year_restrict_late_obj)
doc_topics_df_late

,letter,date,sender,recipient,text,topic
2521,letters_217.xml,1843-11-22,"Henslow, J. S.","Hooker, J. D.",suffolk,0_museum_ipswich_lecture_suffolk
2622,letters_230.xml,1845-02-06,"Henslow, J. S.","Whewell, William",appointment curator bigg progress arrangement ...,0_museum_ipswich_lecture_suffolk
2629,letters_232.xml,1845-11-15,"Henslow, J. S.","Hooker, J. D.",congratulate arnott appointment chair appointm...,0_museum_ipswich_lecture_suffolk
2636,letters_234.xml,1846-02-28,"Henslow, J. S.","Hooker, J. D.",cambridge week home saturdays april,0_museum_ipswich_lecture_suffolk
2672,letters_242.xml,1848-04-04,"Henslow, J. S.","Babbage, Charles",museum worth steam kew garden superintendence,0_museum_ipswich_lecture_suffolk
...,...,...,...,...,...,...
8244,letters_1161.xml,1846-02-12,"Coleman, Henry","Henslow, J. S.",duty christian mite treasury effect,14_church_sabbath_parishioner_clergyman
8247,letters_1161.xml,1846-02-12,"Coleman, Henry","Henslow, J. S.",street worship form exception conduct inhabitant,14_church_sabbath_parishioner_clergyman
8248,letters_1161.xml,1846-02-12,"Coleman, Henry","Henslow, J. S.",poor masse church chapel fact class,14_church_sabbath_parishioner_clergyman
8252,letters_1161.xml,1846-02-12,"Coleman, Henry","Henslow, J. S.",scoff religion sing praise belly,14_church_sabbath_parishioner_clergyman


Use all_tree_vis function to create dendrogram visualisation of clusters for most prominent topics, with dataframe and topic model as parameters.

In [20]:
all_tree_vis = bertopic_tree(sent_df, topic_model)
all_tree_vis

Use year_counts and year_counts_plot functions to get a count of letters in each year and visualise.

In [21]:
year_counts_all = year_counts(df)
letter_nums = year_counts_plot(year_counts_all)
letter_nums

Create visualisation of topics over time for letters sent by Henslow using bertopic_time function.

In [22]:
henslow_vis_obj = bertopic_time(henslow_sent_df, topic_model)
henslow_vis = henslow_vis_obj[0]
henslow_vis

Use year_counts and year_counts_plot functions to get a count of letters from Henslow in each year and visualise.

In [23]:
year_counts_henslow = year_counts(henslow_df)
letter_nums_hen = year_counts_plot(year_counts_henslow)
letter_nums_hen

Create visualisation of topics over time for letters sent to Henslow using bertopic_time function.

In [24]:
non_henslow_vis_obj = bertopic_time(non_henslow_sent_df, topic_model)
non_henslow_vis = non_henslow_vis_obj[0]
non_henslow_vis

Use year_counts and year_counts_plot functions to get a count of letters to Henslow in each year and visualise.

In [25]:
year_counts_non_henslow = year_counts(non_henslow_df)
letter_nums_non_hen = year_counts_plot(year_counts_non_henslow)
letter_nums_non_hen

Use year_window_vis function to create a visualisation of topics over time, omitting years with very high or low numbers of letters.

In [26]:
all_win_vis = year_window_vis(sent_df, year_counts_all, topic_model)
all_win_vis

Use year_window_vis function to create a visualisation of topics over time for letters sent by Henslow, omitting years with very high or low numbers of letters.

In [27]:
henslow_win_vis = year_window_vis(henslow_sent_df, 
                                  year_counts_henslow, 
                                  topic_model, 
                                  low_lim=10, 
                                  high_lim=150
                                 )
henslow_win_vis

Use year_window_vis function to create a visualisation of topics over time for letters sent to Henslow, omitting years with very high or low numbers of letters.

In [28]:
non_henslow_win_vis = year_window_vis(non_henslow_sent_df, 
                                      year_counts_non_henslow, 
                                      topic_model, 
                                      low_lim=10, 
                                      high_lim=150
                                     )
non_henslow_win_vis

Save visualisations to html files

In [29]:
all_vis.write_html("images/all_letters/bertopic_noun_propn.html")
all_win_vis.write_html("images/all_letters/bertopic_noun_propn_window.html")
all_tree_vis.write_html("images/all_letters/bertopic_topic_tree.html")

sent_df_year_restrict_20_vis.write_html("images/year_window/bertopic_20s.html")
sent_df_year_restrict_late_vis.write_html("images/year_window/bertopic_1845_1857.html")

henslow_vis.write_html("images/from_henslow/from_henslow.html")
henslow_win_vis.write_html("images/from_henslow/from_henslow_window.html")

non_henslow_vis.write_html("images/to_henslow/to_henslow.html")
non_henslow_win_vis.write_html("images/to_henslow/to_henslow_window.html")

letter_nums.write_html("images/all_letters/year_count.html")
letter_nums_hen.write_html("images/from_henslow/year_count_from_henslow.html")
letter_nums_non_hen.write_html("images/to_henslow/year_count_to_henslow.html")

Save dataframes containing topic names for visualisations for use in social network analysis

In [30]:
doc_topics_df.to_pickle("pickle/documents_topics.pkl")
doc_topics_df_20s.to_pickle("pickle/documents_topics_20s.pkl")
doc_topics_df_late.to_pickle("pickle/documents_topics_late.pkl")